<a href="https://colab.research.google.com/github/cbroms/a-society/blob/master/We_Live_In_a_Society.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We have a world of citizens that can form societies.  Each citizen has a certain well-defined amount of **skill** which is initialized to be random and
which can change.

Citizens can either join an existing society or form a new one.  Each citizen has a certain amount of **desire for social connectivity**, also intialized to be random.  If a citizen has a greater **desire for social connectivity** they will evaluate which society they want to be a part of more. If a citizen's resources are below a minimum level, then their desire for social connectivity increases, making it easier for them to move around. 

 Citizens can choose societies based on how much **skill** the society has: this is the sum of the **skill** of its existing citizens.  If a society has more **skill**, they are able to produce essential resources more efficiently (for now there is only one).  Furthermore, a citizen can grow their own **skill** more quickly in a society with more **skill**. However, in a society with more **skill**, there is a greater chance that the citizen will be of a lower rank and will therefore receive fewer of the created resources.  A society has **N** ranks according to its size, and a citizen is given a **rank** based on their **skill** using a Power Law distribution.

Resource distribution is linear (for now, or it can be based on society) relative to rank (if there are **N** ranks and a citizen is of rank 2, they get **N-2** resources on every allocation).

Things To Consider:
- how do citizens use their resources?
- how to choose society to go to?
- how does resources depend on skill-level of society?
- citizens decide themselves how they learn (should it be changed via a constant
  based on the skill/size of the society? or based on rank? => or treat   increased_skill as a resource)
- do societies distribute all resources or save some?
- for later, each society has different rate of distribution
- citizen has a sense of proximity so that it only evaluates societies it is "close" to

In [0]:
from functools import reduce
from random import randint

In [0]:
class Citizen:
  def constructor(name):
    self.name = name 
    # social score -> lower = easier to move around, higher = harder to move around
    self.base_social = float(randint(10, 20))
    self.desperate_social = float(randint(3,7))
    self.social = self.base_social
    self.look_around_p = 1 / self.social
    # skill level -> defines ability to produce resources and rank within a society
    self.skill = randint(1, 20)
    # minimum resources people generally need (could do Gaussian around 15)
    self.min_resources = randint(13, 17)
    # the resources owned by a citizen (around 0 - 100 at a time)
    self.resources = 0
    # society it is in
    self.society = None
    self.societal_rank = None

  def update_resource(amount):
    self.resources += amount
    if (self.resources < self.min_resources): 
      self.social = self.desperate_social
    else:
      self.social = self.base_social
    self.look_around_p = 1 / self.social

  def look_around(societies):
    assert(len(societies) < 100) # or number of citizens

    societies = filter((lambda x: x.name == self.society.name), societies)
    chosen = max([(s.predict_resources(self), s) for s in societies])

    # predict_resources should check if member is in or not (for next round)
    if chosen[0] > self.society.predict_resources(self):
      if random() < self.look_around_p: 
        return chosen[1]
      
    return None
    #look at all the possible societies and decide if it wants to move
    # how does it decide to move? 
    # return None if not going to move, otherwise return the target society

  def update_skill(added_skill):
    # change the citizens' skill levels 
    #for citizen in self.members:
     citizen.skill += added_skill

  def use_resource():
    self.resources -= 1

In [0]:
class Society:
  def constructor(name):
    self.name = name
    self.members = []
    # the overall skill of the society
    self.skill = 0
    self.resources = 0
    # for Power distribution in resource allocation
    self.d = random() * 15 + 5 # [5,20]  
    self.h = random() + 2 # [2,3] 
    self.distribute = lambda r: 0 # initialized

  def distribute_generator(size):
    return lambda r: (size / self.d) * pow(r, self.h) 

  def update_skill(added_skill):
    #self.skill = reduce((lambda x, y: x.skill + y.skill), self.members)
    self.skill += added_skill

  def add_member(citizen):
    citizen.society = self
    self.members.append(citizen)
    self.distribute = distribute_generator(len(self.members))
    update_skill(citizen.skill)

  def remove_member(citizen):
    filter((lambda x: x.name == citizen.name), self.members)
    self.distribute = distribute_generator(len(self.members))
    update_skill(-citizen.skill)
    citizen.society = None

  # update ranking of members
  def update_ranking():
    rel_rank = sorted([(m.skill, m) for m in self.members])[::-1]
    for i, (_,m) in enumerate(rel_rank):
      m.societal_rank = i
    return

  def distribute(attribute, update):
    # assume members have perfect linear rank order

    # apply function to every person to get relative proportion
    rel_prop = [(self.distribute(m.societal_rank),m) for m in self.members]
    norm = sum([p for (p,m) in rel_prop])
    # TODO: give out all the created resources
    rec_alloc = [(p/norm * attribute, m) for (p,m) in rel_prop]
    for (r,m) in rel_prop:
      m.update(r)
    return

  def create_and_distribute_resources():
    # based on # of members and skill
    self.resources += len(self.members) * self.skill # TODO: maybe have const
    self.distribute(self.resources, update_resource) # HIGHLY FISHY func pass

  def create_and_distribute_skill():
    added_skill += self.skill / len(self.members) # TODO: maybe have const
    self.distribute(added_skill)
    update_skill(added_skill, update_skill) # HIGHLY FISHY func pass

In [0]:
class World:
  def constructor():
    self.societies = []
    # the overall wealth of the world 
    self.wealth = 0

  def let_members_move():
    # for every member of the societies, let them decide if they want to move or 
    # stay and then do that action 
    for society in self.societies:
      for citizen in society.members:
        move = citizen.look_around(self.societies)
        if (move):
          society.remove_member(citizen)
          move.add_member(citizen)

  def run_model():
    for society in self.societies:
      society.create_and_distribute_skill()
      society.update_ranking()
      society.create_and_distribute_resources()

    self.let_members_move()

  def print_world():
    for society in self.societies:
      print("Society: " + society.name)
      for citizen in society.citizens:
        print("Rank " + citizen.rank + ": Citizen " + citizen.name + ", Resources: " + citizen.resources + ", Skill: " + citizen.skill)
      print("/n")

In [0]:
this_world = World()
citizens = []
societies = []

# N: number of "threads"
for i in range(0, 100):
  citizen = Citizen(i)
  society = Society(i)
  society.add_member(citizen)
  citizens.append(citizen)
  societies.append(society)

# T: timesteps
for i in range(0, 0):
  this_world.run_model()

this_world.print_world()




TypeError: ignored